<a href="https://colab.research.google.com/github/kavyajeetbora/end_to_end_gee_with_python/blob/master/notebooks/14_cloud_masking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cloud Masking

In Google Earth Engine, masking pixels in an image makes those pixels transparent and excludes them from analysis. Here’s how it works:

Image Masks:
- Each pixel in each band of an image has a mask.
- Pixels with a mask value of 0 or below are transparent.
- Pixels with a mask value above 0 are rendered.
- You can set an image’s mask using `image1.mask(image2)`

In this tutorial we will mask the clouds from image collection of:
- sentinel
- landsat

In [25]:
import ee
import xarray
import geemap

ee.Authenticate()
ee.Initialize(project='kavyajeetbora-ee')

In [5]:
admin = ee.FeatureCollection('FAO/GAUL_SIMPLIFIED_500m/2015/level2')

sonitpur = admin.filter(ee.Filter.eq('ADM1_NAME', 'Assam'))\
.filter(ee.Filter.eq('ADM2_NAME', 'Sonitpur'))

geometry = sonitpur.geometry()

## Sentinel

In [28]:
s2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')

filtered_img = s2.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',30))\
.filter(ee.Filter.date('2019-01-01','2020-01-01'))\
.filter(ee.Filter.bounds(geometry))

filtered_img.size()

Masking the following classes to 0:
- 3: Cloud Shadows
- 8: Clouds Medium Probability
- 9: Clouds High Probability

In [29]:
def cloud_free_sentinel(image):
    scl = image.select('SCL')
    mask = scl.eq(3).Or(scl.gte(7).And(scl.lte(10)))
    return image.updateMask(mask.eq(0))

ic = filtered_img.map(cloud_free_sentinel)

### Visualize the results

In [31]:
Map = geemap.Map()
viz_params = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}
Map.addLayer(ic.first(), viz_params, 's2')
Map.centerObject(geometry, zoom=10)
Map

Map(center=[26.76086472764834, 92.93095436904682], controls=(WidgetControl(options=['position', 'transparent_b…

## Landsat images

In this case we will mask out these following classes:
- Bit 1: Dilated Cloud
- Bit 2: Cirrus (high confidence)
- Bit 3: Cloud
- Bit 4: Cloud Shadow

**Merging Landsat 8 and Landsat 9 image collections**: is beneficial for several reasons:

1. **Enhanced Temporal Coverage**:
   - Combining Landsat 8 and Landsat 9 data increases the revisit time for a given area.
   - Landsat 8 + Landsat 9 revisit time is approximately every 8 days, similar to Landsat 8 + Landsat 7⁴.
   - More frequent observations allow better monitoring of land changes, vegetation dynamics, and natural disasters.

2. **Improved Data Availability**:
   - Landsat 9, like Landsat 8, has a higher imaging capacity than previous Landsat satellites.
   - Merging both collections results in around 1,400 scenes per day added to the Landsat global land archive⁴.
   - Researchers and analysts benefit from a larger dataset for various applications.

3. **Consistent Data Processing**:
   - By merging the collections, you ensure consistent data processing and analysis.
   - Common preprocessing steps (e.g., atmospheric correction, cloud masking) can be applied uniformly.

Remember that combining Landsat 8 and Landsat 9 data provides a more comprehensive dataset for Earth observation and land monitoring. Let me know if you need further details! 😊⁴

Source: Conversation with Bing, 17/5/2024
(1) Landsat 9 | Landsat Science - NASA. https://landsat.gsfc.nasa.gov/satellites/landsat-9/.
(2) Merging Landsat into imagecollection (Landsat 7 8 9) using ... - YouTube. https://www.youtube.com/watch?v=pv1cF9NFNXA.
(3) How To Download Landsat 9 & Landsat 8 Level 1 & 2 Image Free!!. https://www.youtube.com/watch?v=HmdKW2h2yAA.
(4) Landsat 8 Image Classification with ArcGIS (Supervised). https://www.youtube.com/watch?v=x8uFTix3sHU.
(5) Merging raw data from Landsat 5, Landsat 7 and Landsat 8. https://gis.stackexchange.com/questions/338915/merging-raw-data-from-landsat-5-landsat-7-and-landsat-8.
(6) Merge ImageCollections with different pixel size. https://gis.stackexchange.com/questions/329735/merge-imagecollections-with-different-pixel-size.
(7) undefined. https://cutt.ly/5CEuzSD.

In [45]:
lc9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
lc8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")

LC = lc8.merge(lc9)

filtered_img = LC.filter(ee.Filter.lt('CLOUD_COVER',30))\
.filter(ee.Filter.date('2021-01-01','2024-01-01'))\
.filter(ee.Filter.bounds(geometry))

filtered_img.size()

In [46]:
def cloud_free_landsat(image):
    qa = image.select('QA_PIXEL')

    dilated = 1 << 1
    cirrus = 1 << 2
    cloud = 1 << 3
    shadow = 1 << 4

    mask = qa.bitwiseAnd(dilated).eq(0)\
    .And(qa.bitwiseAnd(cirrus).eq(0))\
    .And(qa.bitwiseAnd(cloud).eq(0))\
    .And(qa.bitwiseAnd(shadow).eq(0));

    return image.updateMask(mask)

In [50]:
lc = filtered_img.map(cloud_free_landsat)
lc_composite = lc.median().multiply(2.75e-05).add(-0.2)

### Visualize the results

In [51]:
Map = geemap.Map()
viz_params = {
    'min': 0,
    'max': 0.3,
    'bands': ['SR_B4', 'SR_B3', 'SR_B2']
}
Map.addLayer(lc_composite, viz_params, 's2')
Map.centerObject(geometry, zoom=10)
Map

Map(center=[26.76086472764834, 92.93095436904682], controls=(WidgetControl(options=['position', 'transparent_b…